In [1]:



import transformers


In [2]:
import torch
from typing import Any
from dataclasses import dataclass

LLMModel = Any


# TODO: Cite https://github.com/abazarova/tda4hallucinations/
@dataclass
class TokenwiseEntropy:
    llm_model: LLMModel
    device: str = "cuda"

    @torch.no_grad()
    def calculate(self, input_ids) -> float:
        token_distribution = self._get_token_distribution(input_ids)
        entropy = self._compute_entropy_from_logits(token_distribution)
        return entropy.detach().cpu().item()

    def _get_token_distribution(self, input_ids) -> torch.Tensor:
        # Yield the output of the model for the current example
        output = self.llm_model(
            input_ids,
            output_hidden_states=True,
            output_attentions=False,
        )

        return output.logits[0, -1:]

    def _compute_entropy_from_logits(self, logits: torch.Tensor) -> torch.Tensor:
        """
        Compute entropy from logits.

        Parameters:
        ----------
        logits : torch.Tensor
            Logits from the model.

        Returns:
        -------
        torch.Tensor
            Entropy values.
        """
        probabilities = torch.softmax(logits, dim=-1)
        log_probabilities = torch.log(probabilities + 1e-12)
        entropies = -torch.sum(probabilities * log_probabilities, dim=-1)
        return entropies[0]

In [3]:
from typing import List


def get_sys_prompt(subject: str | None = None):
    if subject is not None:
        sys_msg = f"The following are multiple choice questions about {subject}."
    else:
        sys_msg = "The following are multiple choice questions."

    sys_msg += " Write only the answer number and nothing else."
    return sys_msg


option_ids = [str(i + 1) for i in range(20)]


def get_user_prompt(question: str, options: List[str]):
    options_str = "\n".join([f"{option_id}. {answer}".strip() for option_id, answer in zip(option_ids, options)])
    user_prompt = f"Question: {question.strip()}\nOptions:\n{options_str}\nChoose one of the answers. Write down ONLY the NUMBER of the correct answer and nothing else."
    return user_prompt


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from tokenizers import Tokenizer
# from tokenizers.models import BPE
# from tokenizers.pre_tokenizers import Whitespace

MODEL_NAME = "Mistral-Small-24B-Instruct-2501"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# tokenizer = Tokenizer(BPE())
# tokenizer.pre_tokenizer = Whitespace()


model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto", trust_remote_code=True,) 

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [6]:
print(model.get_memory_footprint() / 10**9)
print(next(model.parameters()).device)

94.28962304
cuda:0


In [5]:
if torch.cuda.is_available():
    DEVICE = torch.device("cuda:2")

In [6]:
!nvidia-smi

Thu Feb  6 22:18:33 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000000:0B:00.0 Off |                    0 |
| N/A   35C    P0              81W / 400W |  44394MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [7]:
import pandas as pd
import ast
import csv
from tqdm import tqdm
import gc
import os, sys


DUMP_EVERY = 100
invalid_answers = 0


def estimate_dataset(
    df, model, tokenizer, get_subject_from_row, get_question_from_row, get_options_from_row, verify_answer, out_filename
):
    global invalid_answers

    model_name = model.config_class().model_type
    print(model_name)

    field_ans_correct = f"entropy_ans_correct_{model_name}"
    field_ans_output = f"entropy_ans_output_{model_name}"
    field_entropy_value = f"field_entropy_value_{model_name}"

    if field_ans_correct not in df.columns:
        df[field_ans_correct] = False
    if field_ans_output not in df.columns:
        df[field_ans_output] = ""
    if field_entropy_value not in df.columns:
        df[field_entropy_value] = 0.0

    entropy_estimator = TokenwiseEntropy(llm_model=model, device=DEVICE)

    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        if df.at[index, field_ans_output] != "" and df.at[index, field_ans_output] != "INVALID":
            continue

        print(f"loop {index} -> start: {model.get_memory_footprint(return_buffers=True) / 10**9} GB")

        sys_prompt = get_sys_prompt(get_subject_from_row(row))
        user_prompt = get_user_prompt(get_question_from_row(row), get_options_from_row(row))
        messages = [
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": user_prompt},
        ]
        formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

        inputs = tokenizer(formatted_prompt, return_tensors="pt").to(DEVICE)

        outputs = model.generate(**inputs, max_new_tokens=1, pad_token_id=tokenizer.eos_token_id)
        print(f"loop {index} -> after generate: {model.get_memory_footprint(return_buffers=True) / 10**9} GB")

        input_length = inputs.input_ids.shape[1]
        answer_raw = outputs[0, input_length:]
        answer = tokenizer.decode(answer_raw, skip_special_tokens=True)
        if answer in option_ids:
            entropy = entropy_estimator.calculate(outputs)
            print(f"loop {index} -> after entropy: {model.get_memory_footprint(return_buffers=True) / 10**9} GB")
            df.at[index, field_entropy_value] = entropy
            df.at[index, field_ans_output] = answer
            df.at[index, field_ans_correct] = verify_answer(row, answer)
        else:
            df.at[index, "entropy_ans_output"] = "INVALID"
            invalid_answers += 1

        print(f"Answer: {answer}\nEntropy: {entropy}\nis_correct: {df.at[index, field_ans_correct]}\n\n")

        if index % DUMP_EVERY == 0:
            df.to_csv(out_filename, sep="\t", quoting=csv.QUOTE_NONE, quotechar="", escapechar="\\", index=False)

        gc.collect()
        torch.cuda.empty_cache()

    df.to_csv(out_filename, sep="\t", quoting=csv.QUOTE_NONE, quotechar="", escapechar="\\", index=False)
    print(f"Processed dataset {out_filename}. Total entries: {df.shape[0]}. Invalid answers: {invalid_answers}")
    return df


ORIGINAL_DATASET = "./mmlu_pro_stem"
original_filename = f"{ORIGINAL_DATASET}.tsv"
out_filename = f"{ORIGINAL_DATASET}_w_mistral_entropy_tail.tsv"

if os.path.isfile(out_filename):
    df = pd.read_csv(
        out_filename,
        sep="\t",
        header=0,
        quoting=csv.QUOTE_NONE,
        quotechar="",
        escapechar="\\",
    ).tail(6016)
else:
    df = pd.read_csv(
        original_filename,
        sep="\t",
        header=0,
    ).tail(6016)
# df = df.head(10)


def verify_model_answer(row, model_answer):
    try:
        return int(row["answer_index"]) + 1 == int(model_answer)
    except:
        return False


estimate_dataset(
    df=df,
    model=model,
    tokenizer=tokenizer,
    get_subject_from_row=lambda row: row["base_cluster"],
    get_question_from_row=lambda row: row["question"],
    get_options_from_row=lambda row: ast.literal_eval(row["options"]),
    verify_answer=verify_model_answer,
    out_filename=out_filename,
)

mistral


  0%|          | 0/6016 [00:00<?, ?it/s]

loop 6016 -> start: 94.28962304 GB
loop 6016 -> after generate: 94.28962304 GB


  0%|          | 1/6016 [00:03<5:43:26,  3.43s/it]

loop 6016 -> after entropy: 94.28962304 GB
Answer: 2
Entropy: 0.21088853478431702
is_correct: False


loop 6017 -> start: 94.28962304 GB
loop 6017 -> after generate: 94.28962304 GB


...

100%|█████████▉| 6015/6016 [4:03:43<00:02,  2.22s/it]

loop 12030 -> after entropy: 94.28962304 GB
Answer: 2
Entropy: 0.3370075225830078
is_correct: True


loop 12031 -> start: 94.28962304 GB
loop 12031 -> after generate: 94.28962304 GB
loop 12031 -> after entropy: 94.28962304 GB
Answer: 2
Entropy: 0.3403957486152649
is_correct: False




100%|██████████| 6016/6016 [4:03:45<00:00,  2.43s/it]

Processed dataset ./mmlu_pro_stem_w_mistral_entropy_tail.tsv. Total entries: 6016. Invalid answers: 326


,src,answer,options,category,question,cot_content,question_id,answer_index,total_tokens,meta_cluster,base_cluster,entropy_ans_correct_mistral,entropy_ans_output_mistral,field_entropy_value_mistral,entropy_ans_output
6016,stemez-Thermodynamics,A,"['89,368.79 kJ', '95,500 kJ', '120,000 kJ', '1...",engineering,Methane is stored at low temperature inside a ...,NaN,11442,0,138,Engineering Calculations,Heat Transfer Problems,False,2,0.210889,NaN
6017,ori_mmlu-professional_psychology,H,"['Loss of control', 'Withdrawal', 'Risking sig...",psychology,Which of the following is not represented in D...,NaN,2214,7,62,Psychology Behavior,Psychology Questions (0),False,6,0.122072,NaN
6018,ori_mmlu-professional_psychology,E,['the girl is more likely than the boy to mani...,psychology,Carol and Jim are experiencing marital difficu...,NaN,2215,4,196,Psychology Behavior,Psychology Questions (0),False,9,0.169940,NaN
6019,ori_mmlu-high_school_european_history,F,['had already started making strategies for th...,history,This question refers to the following informat...,NaN,4828,5,305,Historical Trivia,US History Questions,True,6,0.093749,NaN
6020,ori_mmlu-high_school_european_history,C,"['when the people wish to change governors', '...",history,This question refers to the following informat...,NaN,4829,2,353,Historical Trivia,US History Questions,True,3,0.240667,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12027,ori_mmlu-high_school_macroeconomics,F,['Higher interest rates that result from borro...,economics,"The ""crowding-out"" effect refers to which of t...",NaN,7681,5,150,Economics & Finance MCQs,Economic Concepts & Policies,True,6,0.176050,NaN
12028,ori_mmlu-high_school_macroeconomics,A,['Lower reserve requirements; lower the discou...,economics,Which of the following lists contains only Fed...,NaN,7683,0,124,Economics & Finance MCQs,Economic Concepts & Policies,True,1,0.354294,NaN
12029,ori_mmlu-high_school_macroeconomics,I,['The productivity of labor in country X is 75...,economics,Output in country X is 30000 units and there a...,NaN,7684,8,206,Economics & Finance MCQs,Economic Concepts & Policies,False,8,0.322461,NaN
12030,ori_mmlu-high_school_macroeconomics,B,"['an increase in net exports', 'a decrease in ...",economics,A use of easy money (expansionary) policy by t...,NaN,7685,1,58,Economics & Finance MCQs,Economic Concepts & Policies,True,2,0.337008,NaN
